In [1]:
import pandas as pd
import numpy as np
from twitch import TwitchClient
import twitchly_db

In [2]:
user_channel_info = pd.read_csv('list_of_channels_followed_by_user.txt', delimiter='\t', header=None)
user_channel_info.columns = ['user_id', 'channel_info']
user_channel_info.head()

,user_id,channel_info
0,268234051,"{'mature': False, 'status': '!CUBE IN CHAT FOR..."
1,268234051,"{'mature': False, 'status': 'Morning stream on..."
2,268234051,"{'mature': True, 'status': 'Practice with VicV..."
3,268234051,"{'mature': False, 'status': 'SUB GAMES.. Drunk..."
4,268234051,"{'mature': False, 'status': 'High Kill Solos',..."


### Data in flat file is incomplete, need to be able to access channel_id

Otherwise database would be filled with a bunch of channels that nobody follows (channels associated with followers not streamers)

In [3]:
import re

In [4]:
def get_channel_id(c_info):
    return int(re.search(r', \'id\': \'(\d+)\',', c_info).group(1))

user_channel_info['channel_id'] = user_channel_info['channel_info'].apply(get_channel_id)

In [5]:
user_channel_info.head()

,user_id,channel_info,channel_id
0,268234051,"{'mature': False, 'status': '!CUBE IN CHAT FOR...",29829912
1,268234051,"{'mature': False, 'status': 'Morning stream on...",19571641
2,268234051,"{'mature': True, 'status': 'Practice with VicV...",75264857
3,268234051,"{'mature': False, 'status': 'SUB GAMES.. Drunk...",66691674
4,268234051,"{'mature': False, 'status': 'High Kill Solos',...",60056333


In [6]:
len(user_channel_info['channel_id'].unique())

185745

# Adding relevant channels to Twitchly DB

In [7]:
try:
    db = twitchly_db.Database()
except ValueError:
    db = db

In [9]:
sample_size = 2000

sample_series = user_channel_info['channel_id'].sample(n=sample_size)
responses = []
for idx, channel_id in enumerate(sample_series):
    print(channel_id, idx, end='\r')
    if channel_id:
        response = db.get_user_info(channel_id)
        if response:
            responses.append(dict(response))

In [10]:
channels_df = pd.DataFrame(responses)
channels_df.set_index('display_name', inplace=True)
channels_df.sort_values('followers', inplace=True, ascending=False)
channels_df.drop_duplicates(subset=['id'], inplace=True)
channels_df.to_csv('channels_sample_{}.csv'.format(sample_size))
channels_df.head()

,bio,broadcaster_language,broadcaster_software,broadcaster_type,created_at,description,followers,follows,game,id,...,privacy_options_enabled,private_video,profile_banner,profile_banner_background_color,status,type,updated_at,url,video_banner,views
display_name,,,,,,,,,,,,,,,,,,,,,
Ninja,Professional Battle Royale player. Follow my t...,en,unknown_rtmp,partner,2011-01-16 04:31:20,Professional Battle Royale player. Follow my t...,11884880,"[128479231, 46386566, 214062798, 7054369, 3044...",Fortnite,19571641,...,False,False,https://static-cdn.jtvnw.net/jtv_user_pictures...,NaN,FOOTBALL SUNDAY | Pon Pon merch Live! Teamninj...,user,2018-10-21 18:22:26,https://www.twitch.tv/ninja,https://static-cdn.jtvnw.net/jtv_user_pictures...,354301250
shroud,"Enjoy these highlights/vods, and remember to f...",en,unknown_rtmp,partner,2012-11-03 15:50:32,"Enjoy these highlights/vods, and remember to f...",4681624,"[29140917, 6978352, 132230344, 10406, 54165962...",Call of Duty: Black Ops 4,37402112,...,False,False,https://static-cdn.jtvnw.net/jtv_user_pictures...,NaN,#DoritosBowl Prac #ad @shroud on socials,user,2018-10-25 20:27:52,https://www.twitch.tv/shroud,https://static-cdn.jtvnw.net/jtv_user_pictures...,226571889
TSM_Myth,Keep believing.,en,unknown_rtmp,partner,2015-12-27 19:23:58,Keep believing.,4440552,"[70661496, 186356535, 161719481, 240936336, 16...",Fortnite,110690086,...,False,False,https://static-cdn.jtvnw.net/jtv_user_pictures...,#434343,Quick stream before Tcon || !Twitter : TSM_Myth,user,2018-10-25 20:03:08,https://www.twitch.tv/tsm_myth,https://static-cdn.jtvnw.net/jtv_user_pictures...,62085343
summit1g,I'm a competitive CounterStrike player who lik...,en,unknown_rtmp,partner,2011-12-01 06:33:31,I'm a competitive CounterStrike player who lik...,3320497,"[134433708, 78219897, 26301881, 166841009, 126...",Call of Duty: Black Ops 4,26490481,...,False,False,https://static-cdn.jtvnw.net/jtv_user_pictures...,NaN,"matchmaking, blackout, zombies, EVERYTHING [ ...",user,2018-10-25 20:27:16,https://www.twitch.tv/summit1g,https://static-cdn.jtvnw.net/jtv_user_pictures...,245220630
Tfue,Watch Me Play VideoGames,en,unknown_rtmp,partner,2014-04-02 03:57:20,Watch Me Play VideoGames,3238391,"[106941811, 145151151, 47229052, 130149086, 74...",Fortnite,60056333,...,False,False,https://static-cdn.jtvnw.net/jtv_user_pictures...,#ffffff,High Kill Solos,user,2018-10-21 18:19:35,https://www.twitch.tv/tfue,https://static-cdn.jtvnw.net/jtv_user_pictures...,47709889


In [13]:
channels_df.dtypes

bio                                 object
broadcaster_language                object
broadcaster_software                object
broadcaster_type                    object
created_at                          object
description                         object
followers                            int64
follows                             object
game                                object
id                                  object
language                            object
logo                                object
mature                                bool
name                                object
num_follows                        float64
partner                               bool
privacy_options_enabled               bool
private_video                         bool
profile_banner                      object
profile_banner_background_color     object
status                              object
type                                object
updated_at                          object
url        

In [12]:
channels_df.shape

(1713, 26)